In [1]:
#### IMPORTS ####

# Visualisation et manipulation des données
from collections import Counter
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

# Fichiers
import json
import os
import pickle
import zipfile

# Apprentissage
from sklearn.decomposition import NMF
from sklearn.manifold import TSNE as tsne
from sklearn.neighbors import NearestNeighbors

# Interface
from ipywidgets import Button, Layout
from IPython.display import display

# Carte
from bokeh.io import curdoc
from bokeh.layouts import layout, column
from bokeh.models import ColumnDataSource, Div
from bokeh.models.tools import HoverTool
from bokeh.models.widgets import Slider, Select, TextInput
from bokeh.plotting import figure, show, output_file
import ipywidgets as widgets


In [160]:
#### Chargement des données ####

with zipfile.ZipFile('./whats-cooking-data.zip', 'r') as z:
    
    # cuisines
    file = z.open('cuisines.data', 'r')
    cuisines = pickle.load(file)
    file.close()
    
    # dfs
    file = z.open('dfs.data', 'r')
    dfs = pickle.load(file)
    file.close()
    
    # id_par_points_par_pays
    file = z.open('id_par_points_par_pays.data', 'r')
    id_par_points_par_pays = pickle.load(file)
    file.close()
    
    # id_plats_train
    file = z.open('id_plats_train.data', 'r')
    id_plats_train = pickle.load(file)
    file.close()
    
    # ingredients
    file = z.open('ingredients.data', 'r')
    ingredients = pickle.load(file)
    file.close()
    
    # ingredients_par_points_par_pays
    file = z.open('ingredients_par_points_par_pays.data', 'r')
    ingredients_par_points_par_pays = pickle.load(file)
    file.close()
    
    # model
    file = z.open('model.data', 'r')
    model = pickle.load(file)
    file.close()
    
    # neigh
    file = z.open('neigh.data', 'r')
    neigh = pickle.load(file)
    file.close()
    
    # points_par_pays
    file = z.open('points_par_pays.data', 'r')
    points_par_pays = pickle.load(file)
    file.close()
    
    # points_plats
    file = z.open('points_plats.data', 'r')
    points_plats = pickle.load(file)
    file.close()
    
z.close()


/home/julien/anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator NMF from version 0.21.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/julien/anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator NearestNeighbors from version 0.21.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [12]:
#### genere des couleurs aléatoires ####
def couleur_generer():
    number_of_colors = 20
    color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(number_of_colors)]
    return color


In [144]:
#### affichage de nos recettes sur une carte ####
#retourne la source de tous nos points afin de les afficher 
def ajouter(tout_les_points):
    color = couleur_generer()
    #on crée d'abord notre source à affichera avec un dataframe 
    table = []
    tmp = dict()
    for i in range (len(tout_les_points)):
        mes_points = np.asarray(tout_les_points[i])
        for j in range(len(mes_points)):
            tmp['color'] = color[i];
            tmp['cuisine']= cuisines[i]
            tmp = dict()
            tmp['x'] = mes_points[j][1]
            tmp['y'] = mes_points[j][0]
            tmp['id'] = id_par_points_par_pays[i][j]
            tmp['ingredient']=ingredients_par_points_par_pays[i][j].replace(", ", "\n")
            table.append(tmp)

    df_c = pd.DataFrame(table)
    return df_c.fillna(0)

In [145]:
#affiche les points de tous nos points
def afficherCarte(dataSource):
    #on prépare la carte à afficher 
    TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"
    p = figure(tools=TOOLS,  tooltips = [ ('id', '@id'),('cuisine', '@cuisine'), ('ingrédients', '<pre>@ingredient</pre>')])
    p.title.text = 'Carte des plats par couleur de pays'
    source = ColumnDataSource(dataSource) #on installe notre source pour l'afficher 
    p.circle( 'x' , 'y', source= source, fill_color= 'color' , fill_alpha= 1, line_color=None, size=10 )
    show(p)

In [146]:
#Nécessite la source de tous nos points pour mettre en valeur un de ces points 
def mettre_en_valeur_recette(dataSource, monPoint):
    df_test = dataSource.loc[dataSource['id'] == monPoint]
    return df_test
#affichage de nos points + le point qu'on veut mettre en valeur dans un graphe
def afficherCarte2(DS_tout_les_points, DS_monPoint):
    TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"
    p = figure(tools=TOOLS,  tooltips = [ ('id', '@id'),('cuisine', '@cuisine'), ('ingrédients', '<pre>@ingredient</pre>')])
    p.title.text = 'Carte des plats par couleur de pays'
    source = ColumnDataSource(DS_tout_les_points) #on installe notre source pour l'afficher 
    p.circle( 'x' , 'y', source= source, fill_color= 'color' , fill_alpha= 1, line_color=None, size=10 )
    source2 = ColumnDataSource(DS_monPoint)
    p.circle( 'x' , 'y', source= source2, fill_color= 'color' , fill_alpha= 1, line_color=None, size=30)
    show(p)

In [147]:
#affichage d'un tout nouveau point avec ses coordonnées en parametre 
def afficherNouveauPoint(DS_tout_les_points, coordonnee): #[50 ,20]
    TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"
    p = figure(tools=TOOLS,  tooltips = [ ('id', '@id'),('cuisine', '@cuisine'),('ingrédients', '<pre>@ingredient</pre>')])
    p.title.text = 'Carte des plats par couleur de pays'
    source = ColumnDataSource(DS_tout_les_points) #on installe notre source pour l'afficher 
    p.circle( 'x' , 'y', source= source, fill_color= 'color' , fill_alpha= 1, line_color=None, size=10 )
    p.circle( coordonnee[0] , coordonnee[1],  fill_color= 'red' , fill_alpha= 1, line_color=None, size=30)
    show(p)

In [148]:
def dis_moi_qui_me_ressemble_le_plus(vecteur):
    voisins = neigh.kneighbors([vecteur], return_distance=False)[0]
    cuisines_voisines = []
    test = []
    for voisin in voisins:
        cuisines_voisines.append(dfs.loc[dfs['id'] == id_plats_train[voisin]]['cuisine'].unique()[0])
        test.append(id_plats_train[voisin])
    print(cuisines_voisines, voisins, test)
    occurence_count = Counter(cuisines_voisines)
    return occurence_count.most_common(1)[0][0]

In [149]:
def donne_moi_les_coo_de_ma_nouvelle_recette_en_6D(recette):
    recette_csv = []
    for ingr in ingredients:
        tmp = dict()
        tmp['ingredient'] = ingr
        tmp['val'] = 1 if ingr in recette.value else 0
        tmp['id'] = 'Nouvelle recette'
        recette_csv.append(tmp)

    recette_df = pd.DataFrame(recette_csv)
    recette_util_df = pd.pivot_table(data = recette_df,
                                              values = 'val', index = 'id', columns = 'ingredient')
    return model.transform(recette_util_df)[0]


In [150]:
def donne_moi_les_coo_de_ma_nouvelle_recette_en_2D(vecteur):
    voisins = neigh.kneighbors([vecteur], return_distance=False)[0]
    cuisines_voisines = []
    coo = []
    for voisin in voisins:
        cuisines_voisines.append(dfs.loc[dfs['id'] == id_plats_train[voisin]]['cuisine'].unique()[0])
        coo.append(points_plats[voisin])
    return np.mean(coo, axis=0)


In [151]:
#### Construction de la carte des recettes ####

df_c = ajouter(points_par_pays)


In [152]:
#### Affichage de la carte des recettes ####

afficherCarte(df_c)


In [165]:
nouvelle_recette = widgets.SelectMultiple(
    options=ingredients,
    value=['milk', 'honey', 'sweet potatoes'],
    description='Ingrédients :',
    disabled=False,
    layout = Layout(width='650px', height='200px'),
)
display(nouvelle_recette)


SelectMultiple(description='Ingrédients :', index=(4108, 3314, 5957), layout=Layout(height='200px', width='650…

In [166]:
# Récapitulatif du nouveau plat

print(nouvelle_recette.value)


('milk', 'honey', 'sweet potatoes')


In [167]:
#### Récupération des vecteurs 6 et 2 dimentions

nouvelle_recette_W = donne_moi_les_coo_de_ma_nouvelle_recette_en_6D(nouvelle_recette)
coordonnee = donne_moi_les_coo_de_ma_nouvelle_recette_en_2D(nouvelle_recette_W)


In [168]:
#print(nouvelle_recette_W)
print(dis_moi_qui_me_ressemble_le_plus(nouvelle_recette_W))


['southern_us', 'southern_us', 'french', 'british', 'southern_us', 'irish', 'japanese', 'italian', 'french', 'indian'] [34794 36729 33113 20055 13874 19679 11180  1671 34008 35720] ['49170', '661', '47227', '32613', '25651', '32199', '22613', '11906', '48265', '5467']
southern_us


In [169]:
afficherNouveauPoint(df_c, coordonnee)
